# Import packages

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report


# Prepare the data

In [4]:
# Load data
data = pd.read_csv('rainfall_summer_clim.csv')

# Preprocess data: Map original labels to binary classification
def label_mapping(x):
    if x in [2, 3]:
        return 'wet'
    elif x in [0, 1]:
        return 'not wet'
    else:
        return None

data['Rainfall class'] = data['Rainfall class (0: < 0.5, 1: [0.5 4), 2: [4,8), 3: >= 8 (mm/day)'].apply(label_mapping)

# Split data into features and target
X = data[['Longitude (deg)', 'Latitude (deg)']]
y = data['Rainfall class']

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Test different kernels and class weight configurations

In [5]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
class_weights = [None, 'balanced']

for kernel in kernels:
    for weight in class_weights:
        print(f"\nKernel: {kernel}, Class Weight: {weight}")
        model = SVC(kernel=kernel, class_weight=weight)
        model.fit(X_train_scaled, y_train)
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
        print("Training Performance:")
        print(classification_report(y_train, y_pred_train, zero_division=0))
        print("Test Performance:")
        print(classification_report(y_test, y_pred_test, zero_division=0))


Kernel: linear, Class Weight: None
Training Performance:
              precision    recall  f1-score   support

     not wet       0.75      1.00      0.86       962
         wet       0.00      0.00      0.00       323

    accuracy                           0.75      1285
   macro avg       0.37      0.50      0.43      1285
weighted avg       0.56      0.75      0.64      1285

Test Performance:
              precision    recall  f1-score   support

     not wet       0.75      1.00      0.86       414
         wet       0.00      0.00      0.00       137

    accuracy                           0.75       551
   macro avg       0.38      0.50      0.43       551
weighted avg       0.56      0.75      0.64       551


Kernel: linear, Class Weight: balanced
Training Performance:
              precision    recall  f1-score   support

     not wet       0.82      0.50      0.62       962
         wet       0.31      0.68      0.43       323

    accuracy                           0.55 

# Detailed analysis with RBF kernel and different C and gamma values

In [6]:
print("\nDetailed RBF kernel analysis:")
C_values = [100, 30, 10, 3, 1]
gamma_values = [0.5, 0.1, 0.05]

for C in C_values:
    for gamma in gamma_values:
        print(f"\nC: {C}, Gamma: {gamma}")
        model = SVC(kernel='rbf', C=C, gamma=gamma)
        model.fit(X_train_scaled, y_train)
        y_pred_test = model.predict(X_test_scaled)
        print(classification_report(y_test, y_pred_test, zero_division=0))


Detailed RBF kernel analysis:

C: 100, Gamma: 0.5
              precision    recall  f1-score   support

     not wet       0.85      0.95      0.90       414
         wet       0.76      0.50      0.60       137

    accuracy                           0.83       551
   macro avg       0.80      0.72      0.75       551
weighted avg       0.83      0.83      0.82       551


C: 100, Gamma: 0.1
              precision    recall  f1-score   support

     not wet       0.82      0.96      0.89       414
         wet       0.75      0.38      0.50       137

    accuracy                           0.81       551
   macro avg       0.79      0.67      0.70       551
weighted avg       0.81      0.81      0.79       551


C: 100, Gamma: 0.05
              precision    recall  f1-score   support

     not wet       0.75      1.00      0.86       414
         wet       0.00      0.00      0.00       137

    accuracy                           0.75       551
   macro avg       0.38      0.50   